In [1]:
import pandas as pd

In [2]:

data = pd.read_csv('Sarah_Core.csv')
PR_list = [f'I10_PR{i}' for i in range(1,26)]
data = data[['DRG'] + PR_list]
data.head()

/var/folders/r9/p4wm07b97132b2lm9g78p_7r0000gn/T/ipykernel_82343/1803564337.py:1: DtypeWarning: Columns (10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('Sarah_Core.csv')


,DRG,I10_PR1,I10_PR2,I10_PR3,I10_PR4,I10_PR5,I10_PR6,I10_PR7,I10_PR8,I10_PR9,...,I10_PR16,I10_PR17,I10_PR18,I10_PR19,I10_PR20,I10_PR21,I10_PR22,I10_PR23,I10_PR24,I10_PR25
0,330,0DTH0ZZ,0DTH0ZZ,0DNU4ZZ,0DNH4ZZ,0DNB4ZZ,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,785,10D00Z1,10D00Z1,0UT70ZZ,4A1HXCZ,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,807,10E0XZZ,10E0XZZ,0KQM0ZZ,4A1HXCZ,3E0DXGC,0U7C7ZZ,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,177,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,536,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# if (data[PR_list].count(axis=1) == data[PR_list].nunique(axis=1, dropna=True)).all():
#     print("✅ Every PCS appears at most once per row across PR columns.")
# else:
#     print("⚠️ There are rows with duplicate PCS codes.")

In [4]:
unique_PCSs = {
    c: set(data[c].dropna().unique())
    for c in PR_list
}
unique_PCSs = sorted(list(set().union(*unique_PCSs.values())))
DRG_list = sorted(list(data['DRG'].unique()))

In [5]:
appearance_freq = pd.DataFrame(columns=['Total'] + unique_PCSs)
PCS_code_freq = pd.DataFrame(columns=['Total'] + unique_PCSs)

from collections import Counter
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
    message=".*previous implementation of stack is deprecated.*"
)


for DRG in tqdm(DRG_list, desc="Processing DRGs"):
    sub = data.loc[data['DRG'] == DRG].drop(columns='DRG')
    DRG_ct = len(sub)
    
    s = sub.stack(dropna=True)  
    code_counts = s.value_counts()  
    row_to_codes = (s.reset_index(level=1, drop=True)        
                      .groupby(level=0)                 
                      .agg(lambda sr: set(sr.values)) )     
    
    row_counter = Counter()
    for code_set in row_to_codes:
        row_counter.update(code_set)
    row_counts = pd.Series(row_counter) 
    
    appearance_freq.loc[DRG, row_counts.index] = row_counts
    appearance_freq.loc[DRG, 'Total']    = DRG_ct
    
    PCS_code_freq.loc[DRG, code_counts.index] = code_counts
    PCS_code_freq.loc[DRG, 'Total']          = int(code_counts.sum())
    
PCS_code_freq.index.name = 'DRG'
appearance_freq.index.name = 'DRG'

PCS_code_freq.to_csv('PCS_code_freq.csv', index=True, header=True)
appearance_freq.to_csv('appearance_freq.csv', index=True, header=True)

Processing DRGs:   0%|          | 0/767 [00:00<?, ?it/s]

In [6]:
first_col = PCS_code_freq.columns[0]
mask = (PCS_code_freq.columns == first_col) | PCS_code_freq.columns.str.startswith('B')
PCS_code_freq_filtered = PCS_code_freq.loc[:, mask]
PCS_code_freq_filtered.to_csv('PCS_code_freq_filtered.csv')

first_col = appearance_freq.columns[0]
mask = (appearance_freq.columns == first_col) | appearance_freq.columns.str.startswith('B')
appearance_freq_filtered = appearance_freq.loc[:, mask]
appearance_freq_filtered.to_csv('appearance_freq_filtered.csv', index=True, header=True)